# REINFORCE with baseline

이전 장의 CartPole 환경의 {ref}`cartpole-reinforce` 절에서 REINFORCE 에이전트의 학습 곡선이 들쭉 날쭉한 것을 확인할 수 있었다.
심층강화학습은 여러 측면에서 불안정하다고 말을 많이 한다. 그 중 한 가지 측면이 학습 불안정성이다.
학습이 불안정한 이유는 여러 가지가 있을 수 있지만, 가장 대표적인 이유는 식 {eq}`reinforce-policy-gradient`의 그레디언트 추정값 $\hat{g}$의 분산이 크기 때문이다.
분산이 크다는 의미는 에피소드를 진행하여 얻은 $\hat{g}$의 값이 때론 엄청 크고 때론 엄청 작다는 것이다.
분산이 큰 이유는 에피소드를 진행할 때마다 얻게 되는 return $G_t$의 분산이 크기 때문이다.
Return $G_t$의 분산이 큰 이유는 매 에피소드마다 진행되는 양상이 다르기 때문인데, 환경 안에 초기 상태 분포, 정책의 분포, 전이 확률 분포 등 다양한 랜덤성이 존재하기 때문이다.

<br>

---

## Baseline을 이용하여 그레디언트의 분산 줄이기

강화학습에서는 그레디언트 추정량의 분산을 줄이기 위해 베이스라인 (baseline) 방법을 많이 사용한다. 분산이 큰 이유는 몇몇 있겠지만, 가장 단순한 이유는 기댓값 안의 확률변수가 갖는 값의 크기가 크기 때문이다. 베이스라인 방법을 사용하면 기댓값을 바꾸지 않으면서 확률변수가 갖는 값의 크기를 줄여줄 수 있다.

$$
\nabla_{\mathbf{\theta}} J(\mathbf{\theta}) \propto \sum_s \mu_{\pi_{\mathbf{\theta}}}(s) \sum_{a} \left( Q^{\pi_{\mathbf{\theta}}}(s,a) - b(s) \right) \nabla_{\mathbf{\theta}} \pi_{\mathbf{\theta}}(a|s),
$$ (policy-gradient-with-baseline)

<br>

여기서 $b(s)$ 행동에 의존적이지 않은 함수가 될 수 있으며, 보통 상태가치함수 $V^{\pi_{\theta}}(s)$이 많이 선택된다. $b(s)$를 빼줬는데도 여전히 비례 관계가 성립하는 이유는 다음과 같이 추가된 항들이 결국 0이기 때문이다.

$$
\begin{matrix}
\sum\limits_a b(s) \nabla_{\mathbf{\theta}}\pi_{\mathbf{\theta}}(a|s) & =&  b(s) \sum\limits_a \nabla_{\mathbf{\theta}}\pi_{\mathbf{\theta}}(a|s) \\
& = & b(s) \nabla_{\mathbf{\theta}} \sum\limits_a \pi_{\mathbf{\theta}}(a|s) \\
& = & b(s) \nabla_{\mathbf{\theta}} 1 \\
& = & 0.
\end{matrix} \quad 
$$

<br>

---

## REINFORCE with baseline

이를 이용하여 $G_t$ 대신 $G_t  - b(s_t)$를 업데이트식에 사용하는 방법을 REINFORCE with baseline이라고 한다.

$$
\mathbf{\theta}_{\text{new}}=\mathbf{\theta}_{\text{old}} + \eta \frac{1}{T+1} \sum\limits_{t=0}^{T}\left(  G_t - b(s_t)\right) \nabla_{\mathbf{\theta}} \log \pi_{\mathbf{\theta}_{\text{old}}}(a_t | s_t).
$$ (reinforce-with-baseline)

<br>

잠시 언급한 것처럼 베이스라인으로 상태가치함수를 많이 사용한다. 하지만 상태가치함수를 직접 계산하기 어렵기 때문에 매개변수를 사용하여 상태가치함수를 모델링하게 된다. 즉, 식 {eq}`reinforce-with-baseline`에서 $b(s_t)$ 대신 $V_{\phi}(s_t)$로 적어줄 수 있다.  이때 $\phi \in \mathbb{R}^{m}$은 상태가치함수 모델의 매개변수이다. 이제 상태가치함수 모델의 매개변수도 업데이트해줘야 하는데 상태가치함수의 정의를 사용하여 업데이트하게 된다. 

```{prf:algorithm} REINFORCE with baseline
:label: REINFORCEwithBaseline

1. 정책 네트워크 $\pi_{\theta}$의 파라미터 $\theta$ 초기화, 가치 네트워크 $V_{\phi}$의 파라미터 $\phi$ 초기화 
2. for _ in range(n_episodes):
3. $\qquad$ $\pi_{\theta}$를 따라 에피소드를 진행하여 $\tau=(s_0, a_0, r_0, s_1, a_1, r_1, \ldots, s_T, a_T, r_T)$ 수집
4. $\qquad$ for $t=0, 1, 2, \ldots, T$:
5. $\qquad\qquad$ $G_t = r_t + \gamma r_{t+1} + \ldots + \gamma^{T-t} r_{T}$
6. $\qquad\qquad$ $\hat{g}_t = (G_t - V_{\phi}(s_t)) \nabla_{\theta} \log \pi_{\theta} (a_t|s_t)$
7. $\qquad$ $\hat{g} = \frac{1}{T+1}\sum\limits_{t=0}^{T}\hat{g}_{t}$
8. $\qquad$ $\theta \leftarrow \theta + \eta_{\theta} \hat{g}$  $\quad$ # Gradient ascent
9. $\qquad$ $\phi \leftarrow \phi - \eta_{\phi} \nabla_{\phi} \frac{1}{T+1} \sum\limits_{t=0}^{T} \left(V_{\phi}(s_t)-G_t \right)^2$  $\quad$ # Gradient descent with MSE.
```

여기서 주의할 점은 베이스라인을 사용할 경우 정책 $\pi_{\mathbf{\theta}}$와 상태가치함수 $V_{\phi}$ 둘 다 학습하게 되는데, 그렇다고 이를 actor-critic 방법이라고 부르지는 않는다. 무슨 이유에서인지 모르겠지만, 가치함수 훈련을 위한 target 값을 bootstraping을 사용하여 얻을 경우에만 actor-critic이라고 부른다.
REINFORCE with baseline에서는 $V_{\phi}(s_t)$에 대한 타겟값으로 $G_t$를 사용하고 있기 때문에 actor-critic 방법이라고 부르지 않는다.

<br>

---

## REINFORCE with baseline 구현

Coming soon!

```{raw} html
<script
   type="text/javascript"
   src="https://utteranc.es/client.js"
   async="async"
   repo="HiddenBeginner/Deep-Reinforcement-Learnings"
   issue-term="pathname"
   theme="github-light"
   label="💬 comment"
   crossorigin="anonymous"
/>
```